In [2]:
import numpy as np
from pulp import *
import cvxpy
import pandas as pd

In [3]:
prob = LpProblem("First LP Problem", LpMinimize)

In [4]:
df = pd.read_excel('/Users/sk186089/Desktop/skoda/Optimization-Python-master/Data/diet - medium.xls'
                   ,nrows=17)

In [5]:
df.columns

Index(['Foods', 'Price/Serving', 'Serving Size', 'Calories',
       'Cholesterol (mg)', 'Total_Fat (g)', 'Sodium (mg)', 'Carbohydrates (g)',
       'Dietary_Fiber (g)', 'Protein (g)', 'Vit_A (IU)', 'Vit_C (IU)',
       'Calcium (mg)', 'Iron (mg)'],
      dtype='object')

In [6]:
food_items = list(df['Foods'])

Creating dictionaries for Cost,Calories, Fat and Carbs

In [7]:
costs = dict(zip(food_items,list(df['Price/Serving'])))
calories = dict(zip(food_items, list(df['Calories'])))
fat = dict(zip(food_items, list(df['Total_Fat (g)'])))
carbs = dict(zip(food_items, list(df['Carbohydrates (g)'])))

In [8]:
food_vars = LpVariable.dicts("Food",
                             food_items,
                             lowBound=0,
                             cat='Continous')

In [9]:
prob += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

In [17]:
# Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 50.0, "FatMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 200.0, "CarbsMaximum"

#Calories
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 1500.0, "CalorieMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 2500.0, "CalorieMaximum"

In [18]:
prob.writeLP("SimpleDietProblem.lp")

In [19]:
prob.solve()

1

In [20]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [21]:
print("Therefore, the optimal (least cost) balanced diet consists of\n"+"-"*110)
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Therefore, the optimal (least cost) balanced diet consists of
--------------------------------------------------------------------------------------------------------------
Food_Roasted_Chicken = 0.063522019
Food_Scrambled_Eggs = 6.6239205
Food__Baked_Potatoes = 4.7967144


In [22]:
print("The total cost of this balanced diet is: ${}".format(round(value(prob.objective),2)))

The total cost of this balanced diet is: $3.21
